    Roman Peerboom  10791523
    Floris de Vries 11710799

In [85]:
import numpy
import simpy
import random
import statistics 

In [236]:
lambda_1 = arrival_rate = 10              # Every ten seconds someone arrives, mean of the interval
time_in_system = 20                       # Mean time in system 
mu = 1/time_in_system                     # Mean time in system 


In [244]:
def source(env, number, lambda_1, counter, time_in_system):
    """Source generates customers randomly"""
    t_list = []
    for i in range(number):
        c = object(env, 'Customer%02d' % i, counter, time_in_system)
        env.process(c)
        t = 1-random.expovariate(-lambda_1)
        t_list.append(t)
        yield env.timeout(t)
        
        
def object(env, name, counter, time_in_system):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        results = yield req

        wait = env.now - arrive

        # We got to the counter
        print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

        tib = 1- random.expovariate(-time_in_system)
        yield env.timeout(tib)
        print('%7.4f %s: Finished' % (env.now, name))


lambda_1 = arrival_rate = 1              # Every ten seconds someone arrives, mean of the interval
time_in_system = 2                       # Mean time in system 
mu = 1/time_in_system                    # Mean time in system
number_of_arrivals = 10000000000
print("\N{greek small letter rho} is: " + str(lambda_1/mu))

env = simpy.Environment()
server = simpy.Resource(env, capacity=1) 
env.process(source(env,number_of_arrivals,lambda_1, server, time_in_system))
env.run(until=1000)


ρ is: 2.0
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 1.0362 Customer01: Here I am
 1.0653 Customer00: Finished
 1.0653 Customer01: Waited  0.029
 2.1522 Customer01: Finished
 3.7509 Customer02: Here I am
 3.7509 Customer02: Waited  0.000
 4.9183 Customer02: Finished
 7.1637 Customer03: Here I am
 7.1637 Customer03: Waited  0.000
 8.2190 Customer03: Finished
10.2542 Customer04: Here I am
10.2542 Customer04: Waited  0.000
11.8571 Customer04: Finished
12.2390 Customer05: Here I am
12.2390 Customer05: Waited  0.000
13.8612 Customer05: Finished
15.7795 Customer06: Here I am
15.7795 Customer06: Waited  0.000
16.8235 Customer06: Finished
17.5645 Customer07: Here I am
17.5645 Customer07: Waited  0.000
18.6889 Customer07: Finished
19.1969 Customer08: Here I am
19.1969 Customer08: Waited  0.000
20.2869 Customer08: Finished
21.0192 Customer09: Here I am
21.0192 Customer09: Waited  0.000
22.8261 Customer09: Finished
23.2887 Customer10: Here I am
23.2887 Customer10: Waited  0.

In [245]:
t_list

NameError: name 't_list' is not defined